In [1]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
spark=(SparkSession.builder.master("local[*]").appName("final").getOrCreate())
sc=spark.sparkContext

22/12/03 14:24:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
df=spark.read.option('header','true').csv('anime.csv',inferSchema=True)
df.show()


22/12/03 14:37:55 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------+--------------------+-----+--------------------+--------------------+------------------------------+-----+--------+--------------------+-----------+--------------------+--------------------+----------------+-----------+---------------+--------------------+------+----------+---------+---------+--------+---------+-------+-------+-------------+--------+--------+--------+--------+--------+-------+-------+-------+-------+-------+
|MAL_ID|                Name|Score|              Genres|        English name|                 Japanese name| Type|Episodes|               Aired|  Premiered|           Producers|           Licensors|         Studios|     Source|       Duration|              Rating|Ranked|Popularity|  Members|Favorites|Watching|Completed|On-Hold|Dropped|Plan to Watch|Score-10| Score-9| Score-8| Score-7| Score-6|Score-5|Score-4|Score-3|Score-2|Score-1|
+------+--------------------+-----+--------------------+--------------------+------------------------------+-----+--------+---

In [19]:
from pyspark.sql.functions import col
from pyspark.sql.types import FloatType

f2= df.withColumn("Score",col("Score").cast("float"))
f2.printSchema()

root
 |-- MAL_ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Score: float (nullable = true)
 |-- Genres: string (nullable = true)
 |-- English name: string (nullable = true)
 |-- Japanese name: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Episodes: string (nullable = true)
 |-- Aired: string (nullable = true)
 |-- Premiered: string (nullable = true)
 |-- Producers: string (nullable = true)
 |-- Licensors: string (nullable = true)
 |-- Studios: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Duration: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Ranked: string (nullable = true)
 |-- Popularity: string (nullable = true)
 |-- Members: double (nullable = true)
 |-- Favorites: integer (nullable = true)
 |-- Watching: integer (nullable = true)
 |-- Completed: integer (nullable = true)
 |-- On-Hold: integer (nullable = true)
 |-- Dropped: integer (nullable = true)
 |-- Plan to Watch: integer (nullable = tr

In [20]:
selcol=['Score','Genres','Type','Episodes','Studios','Source','Rating','Popularity',\
        'Members','Favorites','Watching','Completed','On-Hold','Dropped']
df.select(selcol).show()

+-----+--------------------+-----+--------+----------------+-----------+--------------------+----------+---------+---------+--------+---------+-------+-------+
|Score|              Genres| Type|Episodes|         Studios|     Source|              Rating|Popularity|  Members|Favorites|Watching|Completed|On-Hold|Dropped|
+-----+--------------------+-----+--------+----------------+-----------+--------------------+----------+---------+---------+--------+---------+-------+-------+
| 8.78|Action, Adventure...|   TV|      26|         Sunrise|   Original|R - 17+ (violence...|        39|1251960.0|    61971|  105808|   718161|  71513|  26678|
| 8.39|Action, Drama, My...|Movie|       1|           Bones|   Original|R - 17+ (violence...|       518| 273145.0|     1174|    4143|   208333|   1935|    770|
| 8.24|Action, Sci-Fi, A...|   TV|      26|        Madhouse|      Manga|PG-13 - Teens 13 ...|       201| 558913.0|    12944|   29113|   343492|  25465|  13925|
| 7.27|Action, Mystery, ...|   TV|      

In [24]:
from pyspark.sql.functions import *
df.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() | \
                            col(c).contains('Unknown')|\
                            isnan(c), c 
                           )).alias(c)
                    for c in df.columns]).show()

+------+----+-----+------+------------+-------------+----+--------+-----+---------+---------+---------+-------+------+--------+------+------+----------+-------+---------+--------+---------+-------+-------+-------------+--------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|MAL_ID|Name|Score|Genres|English name|Japanese name|Type|Episodes|Aired|Premiered|Producers|Licensors|Studios|Source|Duration|Rating|Ranked|Popularity|Members|Favorites|Watching|Completed|On-Hold|Dropped|Plan to Watch|Score-10|Score-9|Score-8|Score-7|Score-6|Score-5|Score-4|Score-3|Score-2|Score-1|
+------+----+-----+------+------------+-------------+----+--------+-----+---------+---------+---------+-------+------+--------+------+------+----------+-------+---------+--------+---------+-------+-------+-------------+--------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|     0|   4| 5140|    64|       10563|           51|  37|     516|  309|    12814|     7794|    

22/12/04 00:26:23 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 632030 ms exceeds timeout 120000 ms
22/12/04 00:26:23 WARN SparkContext: Killing executors is not supported by current scheduler.
